# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web e em gravações de encontros ao vivo, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: Se o valor dos produtos for abaixo de um preço limite definido por você, você vai descobrir os produtos mais baratos e atualizar isso em uma planilha.
- Em seguida, vai enviar um e-mail com a lista dos produtos abaixo do seu preço máximo de compra.

- No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites.

### Outra opção:

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- O mesmo para o Buscapé
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra.

In [46]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import os
import re
import pandas as pd
import time
servico = Service(ChromeDriverManager().install())

In [ ]:
# Criando meu navegador


navegador = webdriver.Chrome(service=servico)
def analiseBanidos(lista_termosBanidos,nome):
    tem_banidos = False
    for bani in lista_termosBanidos:
        if bani in nome:
            tem_banidos = True
    return tem_banidos 

def analiseNomeCompleto(lista_nomes,nome):
    tem_todos_termos = True
    for produ in lista_nomes:
        if produ not in nome:
            tem_todos_termos = False
    return tem_todos_termos        
            
def buscaGoogleShopp(navegador,nomeProduto,termos_banidos,preco_mini,preco_max):
    
    lista_termosBanidos = termos_banidos.split(' ')
    lista_nomes = nomeProduto.split(' ')
    resultado = []
    navegador.get('https://www.google.com/')
    navegador.find_element(By.XPATH,'//*[@id="APjFqb"]').send_keys(nomeProduto,Keys.ENTER)
    
    # Fiz um esperador de elemento (força a pagina esperar ate que o elemento carregue completamente)

    while len(navegador.find_elements(By.CLASS_NAME,'YmvwI')) < 1:
        time.sleep(1)

    abas = navegador.find_elements(By.CLASS_NAME,'YmvwI')
    for item in abas:
        if item.text in ['Shopping','Compras']:
            item.click()
            break
        

    listaResultados = navegador.find_elements(By.CLASS_NAME,'KZmu8e')

    for elementoResultado in listaResultados:
    
        # # UMA FORMA ALTERNATIVA PARA BUSCAR ATRIBUTOS DO ELEMENTO PAI OU QUE VEM ACIMA
        # eleabaixo = elementoResultado.find_element(By.CLASS_NAME,'EI11Pd')
        # elePai = eleabaixo.find_element(By.XPATH,'..').get_attribute('href')
        # print(elePai)
        
        nome = elementoResultado.find_element(By.CLASS_NAME,'sh-np__product-title').text.lower()
        
        # filtro banido
        tem_banidos = analiseBanidos(lista_termosBanidos,nome)
        
        # filtro nome verificado 
        tem_todos_termos = analiseNomeCompleto(lista_nomes,nome)
        
        # se tiver valor falso nos dois
        if not tem_banidos and tem_todos_termos:
            preco = elementoResultado.find_element(By.CLASS_NAME,'T14wmb').text.replace('R$ ','').replace('.','').replace(',','.')
            site =  elementoResultado.find_element(By.CLASS_NAME,'E5ocAb').text
            link = elementoResultado.find_element(By.CLASS_NAME,'shntl').get_attribute('href')
            imagem = elementoResultado.find_element(By.TAG_NAME,'img').get_attribute('src')
            imagem = f'<img src="{imagem}" alt="" role="presentation" data-atf="1" width="30" height="30">  '
            try:
                preco = float(preco)
            except:

                preco = re.search(re.compile(r'\d+.\d+'),preco)
                preco = float(preco.group(0))
                
            if preco_mini <= preco <= preco_max:
                resultado.append((imagem + nome,preco,site,link))
                
    
    return resultado

# def buscaBuscape(navegador,nomeProduto,termos_banidos,preco_mini,preco_max):

#     lista_termosBanidos = termos_banidos.split(' ')
#     lista_nomes = nomeProduto.split(' ')
#     resultado = []
#     navegador.get('https://www.buscape.com.br/?og=19220&og=19220&gad_source=1&gclid=CjwKCAjw9cCyBhBzEiwAJTUWNQUbkKh9cRFB6tBRt8Jp3Kfw0MUOvSPISKZ72OLZOpU30oOcTNj_eBoCteoQAvD_BwE')
#     navegador.find_element(By.XPATH,'//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(nomeProduto,Keys.ENTER)
   
#    # Fiz um esperador de elemento (esperar que a pagina carregue total)
#     while len(navegador.find_elements(By.CLASS_NAME,'Select_Select__1HNob')) < 1:
#         time.sleep(1)
        
#     # resultados de itens encontrados
#     listaResultados = navegador.find_elements(By.CLASS_NAME,'Hits_ProductCard__Bonl_')

#     for elementoResultado in listaResultados:
    
#         #UMA FORMA ALTERNATIVA PARA BUSCAR ATRIBUTOS DO ELEMENTO PAI OU QUE VEM ACIMA
#         # eleabaixo = elementoResultado.find_element(By.CLASS_NAME,'EI11Pd')
#         # elePai = eleabaixo.find_element(By.XPATH,'..').get_attribute('href')
        
#         nome = elementoResultado.find_element(By.CLASS_NAME,'Text_MobileLabelXs__dHwGG').text.lower()
        
#         # filtrando palavras que nao deve conter no nome do produto
#         tem_banidos = analiseBanidos(lista_termosBanidos,nome)
        
#         # filtrando so nomes que contem tudo que especifique(4k, LG, MONITOR)
#         tem_todos_termos = analiseNomeCompleto(lista_nomes,nome)
                
#         # se passar com sucesso os dois filtros
#         if not tem_banidos and tem_todos_termos:
#             preco = elementoResultado.find_element(By.CLASS_NAME,'Text_MobileHeadingS__HEz7L').text.replace('R$ ','').replace('.','').replace(',','.')
#             link = elementoResultado.find_element(By.CLASS_NAME,'ProductCard_ProductCard_Inner__gapsh').get_attribute('href')
            
#             try:
#                 # pegando precos em caso de ser numeros de pontos flutuantes
#                 preco = float(preco)
#             except:
#                 # pegando na marra ignorando qualquer outro tipo sem ser digito
#                 preco = re.search(re.compile(r'\d+.\d+'),preco)
#                 preco = float(preco.group(0))
                
#             # comparando se o valor esta no meio do parametros
#             if preco_mini <= preco <= preco_max:
#                 resultado.append((nome,preco,link))
    
    
#     return resultado

tabelaProduto = pd.read_csv('buscas.csv')
tabelaOfertas = pd.DataFrame()

for linha in tabelaProduto.index:
    
    nomeProduto = tabelaProduto.loc[linha,'Nome'].lower()
    termos_banidos = tabelaProduto.loc[linha,'Termos banidos'].lower()
    preco_mini = tabelaProduto.loc[linha,'Preco minimo']
    preco_max = tabelaProduto.loc[linha,'Preco maximo']
    
    # GOOGLE -------------------|
    listaGoogle = buscaGoogleShopp(navegador,nomeProduto,termos_banidos,preco_mini,preco_max)
    
    if listaGoogle:# verifica se e vazia
        
        # faz a tupla virar um dataframe
        tabelaGoogle = pd.DataFrame(listaGoogle, columns = ['Nome','Preco','Site','Link'])
        tabelaOfertas = pd.concat([tabelaOfertas,tabelaGoogle])
        
        display(tabelaOfertas)
        
    else:
        tabelaGoogle = None
        
    # BUSCAPE -------------------|
    # listaBuscaPe = buscaBuscape(navegador,nomeProduto,termos_banidos,preco_mini,preco_max)
    
    # if listaBuscaPe:# verifica se e vazia
        
    #     # faz a tupla virar um dataframe
    #     tabelaBuscaPe = pd.DataFrame(listaBuscaPe, columns = ['Nome','Preco','Link'])
    #     tabelaOfertas = pd.concat([tabelaOfertas,tabelaBuscaPe])
    # else:
    #     tabelaBuscaPe = None   
    
tabelaOfertas['Preco'] = tabelaOfertas['Preco'].apply('R${:,.2f}'.format)        
navegador.quit()       

,Nome,Preco,Site,Link
0,"<img src=""https://encrypted-tbn0.gstatic.com/s...",2699.100000,Magazine Luiza,https://www.google.com/aclk?sa=l&ai=DChcSEwiQ1...
1,"<img src=""https://encrypted-tbn3.gstatic.com/s...",2699.102966,KaBuM!,https://www.google.com/aclk?sa=l&ai=DChcSEwiQ1...


,Nome,Preco,Site,Link
0,"<img src=""https://encrypted-tbn0.gstatic.com/s...",2699.100000,Magazine Luiza,https://www.google.com/aclk?sa=l&ai=DChcSEwiQ1...
1,"<img src=""https://encrypted-tbn3.gstatic.com/s...",2699.102966,KaBuM!,https://www.google.com/aclk?sa=l&ai=DChcSEwiQ1...
0,"<img src=""https://encrypted-tbn0.gstatic.com/s...",1559.901835,Terabyteshop,https://www.google.com/aclk?sa=l&ai=DChcSEwib5...


,Nome,Preco,Site,Link
0,"<img src=""https://encrypted-tbn0.gstatic.com/s...",2699.100000,Magazine Luiza,https://www.google.com/aclk?sa=l&ai=DChcSEwiQ1...
1,"<img src=""https://encrypted-tbn3.gstatic.com/s...",2699.102966,KaBuM!,https://www.google.com/aclk?sa=l&ai=DChcSEwiQ1...
0,"<img src=""https://encrypted-tbn0.gstatic.com/s...",1559.901835,Terabyteshop,https://www.google.com/aclk?sa=l&ai=DChcSEwib5...
0,"<img src=""https://encrypted-tbn3.gstatic.com/s...",683.058140,Amazon.com.br,https://www.google.com/aclk?sa=l&ai=DChcSEwjU7...
1,"<img src=""https://encrypted-tbn0.gstatic.com/s...",769.990000,Amazon.com.br,https://www.google.com/aclk?sa=l&ai=DChcSEwjU7...
2,"<img src=""https://encrypted-tbn0.gstatic.com/s...",839.179420,Amazon.com.br,https://www.google.com/aclk?sa=l&ai=DChcSEwjU7...


In [127]:

display(tabelaOfertas)
tabelaOfertas.to_excel('produtosEncontrados.xlsx',index = False)

,Nome,Preco,Site,Link
0,"<img src=""https://encrypted-tbn0.gstatic.com/s...","R$2,699.10",Magazine Luiza,https://www.google.com/aclk?sa=l&ai=DChcSEwiQ1...
1,"<img src=""https://encrypted-tbn3.gstatic.com/s...","R$2,699.10",KaBuM!,https://www.google.com/aclk?sa=l&ai=DChcSEwiQ1...
0,"<img src=""https://encrypted-tbn0.gstatic.com/s...","R$1,559.90",Terabyteshop,https://www.google.com/aclk?sa=l&ai=DChcSEwib5...
0,"<img src=""https://encrypted-tbn3.gstatic.com/s...",R$683.06,Amazon.com.br,https://www.google.com/aclk?sa=l&ai=DChcSEwjU7...
1,"<img src=""https://encrypted-tbn0.gstatic.com/s...",R$769.99,Amazon.com.br,https://www.google.com/aclk?sa=l&ai=DChcSEwjU7...
2,"<img src=""https://encrypted-tbn0.gstatic.com/s...",R$839.18,Amazon.com.br,https://www.google.com/aclk?sa=l&ai=DChcSEwjU7...


# Abrir html

pip install openpyxl


In [6]:
import tkinter as tk
from tkinter import Canvas, Button, messagebox, ttk,filedialog
from PIL import ImageTk, Image
from funcoes import * # Importar minhas funcoes
import pandas as pd
import fitz  # PyMuPDF - # pip install pymupdf
from PIL import Image, ImageTk
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re
import time

servico = Service(ChromeDriverManager().install())


def analiseBanidos(lista_termosBanidos,nome):
    '''Verifica se tem termos banidos no item encontrado'''
    tem_banidos = False
    for bani in lista_termosBanidos:
        if bani != 'NONE' and bani in nome:
            tem_banidos = True
        
    return tem_banidos 

def analiseNomeCompleto(lista_nomes,nome):
    '''Verifica o item encontrado se contem todo nome do produto passado''' 
    tem_todos_termos = True
    for produ in lista_nomes:
        if produ not in nome:
            tem_todos_termos = False
    return tem_todos_termos        
            
def buscaGoogleShopp(navegador,nomeProduto,termos_banidos,preco_mini,preco_max):

    lista_termosBanidos = termos_banidos.split(' ')
    lista_nomes = nomeProduto.split(' ')
    resultado = []
    navegador.find_element(By.XPATH,'//*[@id="APjFqb"]').send_keys(nomeProduto,Keys.ENTER)
   
    # Fiz um esperador de elemento (força a pagina esperar ate que o elemento carregue completamente)

    while len(navegador.find_elements(By.CLASS_NAME,'YmvwI')) < 1:
        time.sleep(1)

    abas = navegador.find_elements(By.CLASS_NAME,'YmvwI')
    for item in abas:
        if item.text in ['Shopping','Compras']:
            item.click()
            break
        

    listaResultados = navegador.find_elements(By.CLASS_NAME,'KZmu8e')
    try:

        for elementoResultado in listaResultados:
        
            # # UMA FORMA ALTERNATIVA PARA BUSCAR ATRIBUTOS DO ELEMENTO PAI OU QUE VEM ACIMA
            # eleabaixo = elementoResultado.find_element(By.CLASS_NAME,'EI11Pd')
            # elePai = eleabaixo.find_element(By.XPATH,'..').get_attribute('href')
            # print(elePai)
            
            nome = elementoResultado.find_element(By.CLASS_NAME,'sh-np__product-title').text.lower()
            
            # filtro banido
            tem_banidos = analiseBanidos(termos_banidos,nome)
 
            # filtro nome verificado 
            tem_todos_termos = analiseNomeCompleto(lista_nomes,nome)

            # se tiver valor falso nos dois
            if not tem_banidos and tem_todos_termos:

                preco = elementoResultado.find_element(By.CLASS_NAME,'T14wmb').text.replace('R$ ','').replace('.','').replace(',','.')
                site =  elementoResultado.find_element(By.CLASS_NAME,'E5ocAb').text
                link = elementoResultado.find_element(By.CLASS_NAME,'shntl').get_attribute('href')
                imagem = elementoResultado.find_element(By.TAG_NAME,'img').get_attribute('src')
                imagem = f'<img src="{imagem}" alt="" role="presentation" data-atf="1" width="30" height="30">  '
                try:
                    preco = float(preco)
                except:

                    preco = re.search(re.compile(r'\d+.\d+'),preco)
                    preco = float(preco.group(0))
                    
                if preco_mini <= preco <= preco_max:
                    resultado.append((imagem + nome,preco,site,link))

        return resultado            
                    
    except:
        # Caso algo de errado retorna uma lista vazia
        return resultado

    





'''********Criar interface gráfica********'''

root = tk.Tk()
root.title("Interface de Busca")

# Variáveis importantes
largura_root = 750
altura_root = 500
lista_temporaria_valores = []# Armazenar todos os valores recebidos dos usuarios para busca
pos_bt_delete = None
tabelaOfertas = None
centralizar_janela(root,largura_root,altura_root)
root.resizable(False,False)

canvas = Canvas(
    root,
    bg = "#000000",
    height = altura_root,
    width = largura_root,
    bd = 0,
    highlightthickness = 0,
    relief = "ridge")
canvas.place(x = 0, y = 0)

imagem_fundo_root = ImageTk.PhotoImage(Image.open(r"imagens\executavel\base.png"))
background = canvas.create_image(largura_root/2, altura_root/2, image=imagem_fundo_root)


'''********ENTRADA DE TEXTO********'''
# Nome do produto
x,y,w,h = posicionar_OBJ_tela(79,192,248,32,5)
entry_nome_produto = tk.Entry(root,bd=0,bg="#D9D9D9",highlightthickness=0,font='Arial 12',relief='flat')
entry_minimo_img = canvas.create_window(x, y,anchor="nw",width=w,height=h, window=entry_nome_produto)

# Termos banidos
x,y,w,h = posicionar_OBJ_tela(79,250,249,32,5)
entry_termos_banidos = tk.Entry(root,bd=0,bg="#D9D9D9",highlightthickness=0,font='Arial 12',relief='flat')
entry_minimo_img = canvas.create_window(x, y,anchor="nw",width=w,height=h, window=entry_termos_banidos)


# Preco minimo
x,y,w,h = posicionar_OBJ_tela(171,312,150,32,5)
entry_minimo = tk.Entry(root,bd=0,bg="#D9D9D9",highlightthickness=0,font='Arial 12',relief='flat')
entry_minimo_img = canvas.create_window(x, y,anchor="nw",width=w,height=h, window=entry_minimo)
formatar_entry(entry_minimo)

# Preco maximo
x,y,w,h = posicionar_OBJ_tela(171,355,150,32,5)
entry_maior = tk.Entry(root,bd=0,bg="#D9D9D9",highlightthickness=0,font='Arial 12',relief='flat')
entry_maior_img = canvas.create_window(x, y,anchor="nw",width=w,height=h, window=entry_maior)
formatar_entry(entry_maior)


'''********BOTOES********'''

def inserir_para_buscar(produto,termos,min,max):
    '''Vou inserir no final de minha tabela'''
    global lista_temporaria_valores

    treeview_listar_produto.insert('','end',values = (produto,min,max))
    # listo novamente minha tabela
    listar_tree(treeview_listar_produto)
    # salvo essas informaçoes na lista temporaria de busca
    lista_temporaria_valores.append((produto,termos,min,max))
    
    
def add_valores_em_lista():
    '''Adiciona valores recuperados dos entry e coloca em lista de busca'''

    # inicializando meus entry que receberao valores dos usuarios
    nome = entry_nome_produto.get()
    termos = entry_termos_banidos.get()
    v_min = entry_minimo.get()
    v_max = entry_maior.get()

    # Verifica campos vazios - evitando que Nome , Valor minimo ou Valor max seja recuperados vazios
    if nome != '' and '0,00' not in[v_min, v_max]:

        # listo meus valores na fila
        inserir_para_buscar(nome,termos,v_min,v_max)

        entry_nome_produto.delete(0,tk.END)
        entry_termos_banidos.delete(0,tk.END)
        entry_minimo.delete(0,tk.END)
        entry_maior.delete(0,tk.END)
    else:
        messagebox.showinfo("Campo vazio", f"Verifica se preencheu todos os campos!")



# Adicionar botão - Adicionar a lista
imagem_adicionar = ImageTk.PhotoImage(Image.open(r'imagens\executavel\adicionar-bt.png'))
x,y,w,h = posicionar_OBJ_tela(98,415,210,44)
bt_adicionar = Button(canvas, image = imagem_adicionar,borderwidth = 0,background='#FDFCE6',
                     activebackground="#FDFCE6",
                     command=add_valores_em_lista)
bt_adicionar.image = imagem_adicionar
bt_adicionar.place(x=x, y=y, width=w, height=h)

# Adicionar botão - Iniciar busca

def buscar_no_gloogleshopping():
    global lista_temporaria_valores, tabelaOfertas

    if len(lista_temporaria_valores) > 0:
        
        # Criando meu navegador
        navegador = webdriver.Chrome(service=servico)
        navegador.get('https://www.google.com/')

        # Crio um DataFrame com a lista_temporaria_valores para comecar a busca
        tabelaProduto = pd.DataFrame(lista_temporaria_valores,columns=['Produtos','Termos banidos','Preco minimo','Preco maximo'])
        # Rezetar minha tabela temporaria
        lista_temporaria_valores = []

        for linha in tabelaProduto.index:

            
            navegador.maximize_window()

            nome_produto = tabelaProduto.loc[linha,'Produtos'].lower()
            termos_banidos = tabelaProduto.loc[linha,'Termos banidos'].lower() if tabelaProduto.loc[linha,'Termos banidos'] not in '' else 'NONE'
            preco_mini = tabelaProduto.loc[linha,'Preco minimo']
            preco_max = tabelaProduto.loc[linha,'Preco maximo']

            
            # GOOGLE -------------------|
            listaGoogle = buscaGoogleShopp(navegador,nome_produto,termos_banidos,converte_float(preco_mini),converte_float(preco_max))
            
            if listaGoogle:# verifica se encontrou alguma coisa
                
                # Concateno a tabela antiga com a nova
                tabelaGoogle = pd.DataFrame(listaGoogle, columns = ['Produtos','Preco','Site','Link'])
                tabelaOfertas = pd.concat([tabelaOfertas,tabelaGoogle])
            
            else:
                tabelaGoogle = None

          
            navegador.find_element(By.CLASS_NAME,'logo').click()

            
        # Aplico uma formatacao na coluna link (deixo no formato R$ REAL)
        try:
                 
            # Exibir o que encontrei
            navegador.quit()
            exibir_meus_resultados_na_web(tabelaOfertas)
            
        except:
            messagebox.showerror('Encontrei nada','Nada foi encontrado, POXA!')
            navegador.quit()
    else:
        messagebox.showwarning("Ação Necessária",
                               "Você precisa passar primeiro qual produto pretende buscar antes de tentar buscar na web.")


def iniciar_busca():
    # Limpar meu treeview
    limpar_tree(treeview_listar_produto)
    buscar_no_gloogleshopping()


imagem_buscar = ImageTk.PhotoImage(Image.open(r'imagens\executavel\buscar-bt.png'))
x,y,w,h = posicionar_OBJ_tela(388,421,111,36)
bt_buscar = Button(canvas, image = imagem_buscar,borderwidth = 0,background='#FDFCE6',
                     activebackground="#FDFCE6",
                     command=iniciar_busca)
bt_buscar.image = imagem_buscar
bt_buscar.place(x=x, y=y, width=w, height=h)

# Adicionar botão - Salvar
def salvar_tabela_busacada():
    global tabelaOfertas
    if tabelaOfertas:
        caminho_sugerido_salve = filedialog.asksaveasfilename(
                            title="Salve sua tabela de Resultados",
                            filetypes=[("Excel files", "*.xlsx *.xls")],# Irar ler so arquivo em xlsx
                            defaultextension=".xlsx"  # Define a extensão padrão
                        )
        
        if caminho_sugerido_salve:
            try:
                # Estou passando minha tabelaOfertas pra o caminho escolido pelo usuario
                tabelaOfertas.to_excel(caminho_sugerido_salve, index=False)
                messagebox.showinfo('Tudo certo!',f"Tabela salva com sucesso em: {caminho_sugerido_salve}")

            except Exception as e:
                messagebox.showinfo('Opa, Erro!',f"Erro ao salvar o arquivo: {e}")

        else:
            return  #Se o usuário cancelar, finaliza a funcao
    else:
        messagebox.showwarning("Ação Necessária","Você precisa buscar algo no Google Shopping antes de tentar salvar os resultados."
)


imagem_salvar = ImageTk.PhotoImage(Image.open(r'imagens\executavel\salvar-bt.png'))
x,y,w,h = posicionar_OBJ_tela(522,422,71,36)
bt_salvar = Button(canvas, image = imagem_salvar,borderwidth = 0,background='#FDFCE6',
                     activebackground="#FDFCE6",
                     command=salvar_tabela_busacada)
bt_salvar.image = imagem_salvar
bt_salvar.place(x=x, y=y, width=w, height=h)

# Adicionar botão - Carregar
def carregar_tabela_local():
    global lista_temporaria_valores
    #  Busca somente o arquivo no formato Excel
    arquivo_excel_local = filedialog.askopenfilename(title="Selecione o arquivo Excel",
                                               filetypes=[("Excel files", "*.xlsx *.xls")]# Busca só arquivo xlsx
    )
    
    if not arquivo_excel_local:
        return  #Se o usuário cancelar, finaliza a funcao
    
     # Limpar meu treeview
    limpar_tree(treeview_listar_produto)
    # Rezetar minha tabela temporaria
    lista_temporaria_valores = []
    exibir_meus_resultados_na_web(arquivo_excel_local)


imagem_carregar = ImageTk.PhotoImage(Image.open(r'imagens\executavel\carregar-bt.png'))
x,y,w,h = posicionar_OBJ_tela(600,422,71,36)
bt_carregar = Button(canvas, image = imagem_carregar,borderwidth = 0,background='#FDFCE6',
                     activebackground="#FDFCE6",
                     command=carregar_tabela_local)
bt_carregar.image = imagem_carregar
bt_carregar.place(x=x, y=y, width=w, height=h)


# Adicionar botao - Guia pratico
def janela_guia():
    # Função para rolar a visualização com o mouse
    
    def carregar_pagina_pdf(pdf_path, fator_zoom):
            """Carrega uma página de PDF e a converte em uma imagem, aplicando o fator de zoom."""

            # Abri o pdf passado no caminho
            documento_pdf = fitz.open(pdf_path)

            # Carregar a primeira pagina
            pagina = documento_pdf[0]  

            # Aplica o zoom ( aumentando a tamanho da imagem )
            # Estou aplicando um zoom de x / zoom de y
            pixel = pagina.get_pixmap(matrix=fitz.Matrix(fator_zoom, fator_zoom))  
            imagem = Image.frombytes("RGB", [pixel.width, pixel.height], pixel.samples)
            return ImageTk.PhotoImage(imagem)
        

    def rolar_com_mouse(event):
        '''Evento gerado pela rolagem do mouse, contendo informações sobre a direção e intensidade do movimento.<br>
                
        -1 * >> Inverte a direção da rolagem.<br>
        event.delta >> Valor pode ser +positivo ou -negativo dependendo se sobe ou desse a rolagem.<br>
        event.delta / 120 >> Divida o valor de event.delta por 120 para normalizar a rolagem.<br>
        units >> Faz a tela rolar verticalmente'''
        
        canvas_guia.yview_scroll(-1 * int(event.delta / 120), "units")  # Ajusta a rolagem vertical


    # Criar a interface Tkinter
    janela_guia_pratico = tk.Toplevel(root)
    janela_guia_pratico.grab_set()# precisa ser chamado apos a criacao
    janela_guia_pratico.transient(root)  # Manter janela_guia_pratico no topo
    janela_guia_pratico.focus_force()  # Força o foco na janela
    janela_guia_pratico.title("Guia Prático")
    janela_guia_pratico.geometry("850x650")
    centralizar_janela(janela_guia_pratico, 850, 650)
    janela_guia_pratico.resizable(False,False)

    
    # Criar canvas e barra de rolagem
    canvas_guia = tk.Canvas(janela_guia_pratico)
    scrollbar = ttk.Scrollbar(janela_guia_pratico, orient="vertical", command=canvas_guia.yview)
    canvas_guia.config(yscrollcommand=scrollbar.set)

    canvas_guia.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    # Caminho para o arquivo PDF
    pdf_path = r"arquivos\guia_praticos_buscador.pdf"

    # Variável para o fator de zoom inicial
    fator_zoom = 1.4

    # Carregar a única página do PDF
    imagem_pagina = carregar_pagina_pdf(pdf_path, fator_zoom)

    # Mostrar a imagem no canvas_guia
    canvas_guia.create_image(0, 0, anchor="nw", image=imagem_pagina)
    canvas_guia.config(scrollregion=canvas_guia.bbox("all"))  # Define a área de rolagem


    # Associar o evento de rolagem do mouse
    canvas_guia.bind_all("<MouseWheel>", rolar_com_mouse)

    root.wait_window(janela_guia_pratico)


imagem_guia = ImageTk.PhotoImage(Image.open(r'imagens\executavel\guia-bt.png'))
x,y,w,h = posicionar_OBJ_tela(671,17,29,22)
bt_guia = Button(canvas, image = imagem_guia,borderwidth = 0,background='#FDFCE6',
                     activebackground="#FDFCE6",
                     command=janela_guia)
bt_guia.image = imagem_guia
bt_guia.place(x=x, y=y, width=w, height=h)


def deletar_item():
    ''''Deleta a linha selecionada e remove o mesmo valor da lista de busca'''
    global lista_temporaria_valores,linha_selecionada_tb
    
    # Com essa posicao descubro qual e seu index
    index_selecionado = treeview_listar_produto.index(linha_selecionada_tb[0])

    # Estou recuperando da lista selecionada a primeira posicao
    nome_produto_selecionado = treeview_listar_produto.item(linha_selecionada_tb[0],'values')[0].title()

    # Mensagem de confirmacao para deletar
    opcao_mensagem = messagebox.askokcancel("Atenção", f'Deseja deletar o produto " {nome_produto_selecionado} " da Busca?')

    if opcao_mensagem:

        # Pecorre todos os itens da linha_selecionada_tb e deleta um por um
        for item in linha_selecionada_tb:
            treeview_listar_produto.delete(item)

        # Remove um valor dessa lista de busca com base no index recuperado da tabela 
        lista_temporaria_valores.remove(lista_temporaria_valores[index_selecionado])


# Adicionar botao - Delete
imagem_delete = ImageTk.PhotoImage(Image.open(r'imagens\executavel\delete-bt.png'))
x,y,w,h = posicionar_OBJ_tela(676,191,22,18)
bt_delete = Button(canvas, image = imagem_delete,borderwidth = 0,background='#FDFCE6',
                    activebackground="#FDFCE6",
                    command=deletar_item)
bt_delete.image = imagem_delete



'''********Treeview********'''


treeview_listar_produto = ttk.Treeview(root,padding=5)
treeview_listar_produto.place(x=388, y=191, width=283, height=196)



# Adicionar barras de rolagem
scrollbar_y = tk.Scrollbar(treeview_listar_produto, orient=tk.VERTICAL)

# Criar o Treeview e configurá-lo com barras de rolagem
treeview_listar_produto = ttk.Treeview(treeview_listar_produto, yscrollcommand=scrollbar_y.set)
scrollbar_y.config(command=treeview_listar_produto.yview)
scrollbar_y.pack(side=tk.RIGHT, fill=tk.Y)
treeview_listar_produto.pack(fill='both')

def exibir_remover_bt_delete(event):
    '''Exibi ou Remove o botao de deletar linha da tela  de forma dinamica'''
    global pos_bt_delete,linha_selecionada_tb,bt_delete

    # Obtem a linha selecionada do treeview
    linha_selecionada_tb = treeview_listar_produto.selection()
    
    # Verifica:
    # 1. Se há uma linha selecionada (linha_selecionada_tb não está vazia)
    # 2. Se a seleção atual é diferente da armazenada em pos_bt_delete 
    if linha_selecionada_tb and pos_bt_delete != linha_selecionada_tb:

        # coloco o botao delete na tela
        bt_delete.place(x=x, y=y, width=w, height=h)

        # Atribuo o mesmo valor da linha selecionada na variavel global
        pos_bt_delete = linha_selecionada_tb

    else:
        # Lista todos os valores na tabela
        listar_tree(treeview_listar_produto)
        # Oculta o botão de deletar
        bt_delete.place_forget()
        # o pos_delete volta a seu valor padrao que é VAZIO
        pos_bt_delete = None


treeview_listar_produto.bind('<<TreeviewSelect>>',exibir_remover_bt_delete)

#Limpa os valores da treeview_listar_produto

listar_tree(treeview_listar_produto)

# Iniciar a interface
root.mainloop()


In [ ]:
p = pd.read_excel('resultados1.xlsx')

p

In [4]:

def remove_formatacao_preco(preco):
    '''Remove todos os caracteres que não são números, ponto, vírgula ou sinal negativo e converte para ponto flutuante'''
    
    # Converte o valor para string
    preco = str(preco)
   
    # Remove todos os caracteres que não são números, ponto, vírgula ou sinal negativo
    # preco = re.sub(r'[^\d,.-]', '', preco)

    # Substitui o ponto (separador de milhares) por uma string vazia e ajusta a vírgula como decimal
    return float(preco.replace('.', '_').replace(',', '.').replace('R$','').replace('_', ''))

print(remove_formatacao_preco('1.000,21'))

1000.21


In [5]:
display(tabelaOfertas)

,Produtos,Preco,Site,Link
0,"<img src=""https://encrypted-tbn1.gstatic.com/s...",R$1.37,Magazine Luiza,"<a href=""<a href=""https://www.google.com/aclk?..."
1,"<img src=""https://encrypted-tbn2.gstatic.com/s...",R$1.46,Mercado Livre,"<a href=""<a href=""https://www.google.com/aclk?..."
2,"<img src=""https://encrypted-tbn2.gstatic.com/s...",R$1.38,AliExpress,"<a href=""<a href=""https://www.google.com/aclk?..."
3,"<img src=""https://encrypted-tbn2.gstatic.com/s...",R$1.50,KaBuM!,"<a href=""<a href=""https://www.google.com/aclk?..."
4,"<img src=""https://encrypted-tbn3.gstatic.com/s...",R$1.38,Mercado Livre,"<a href=""<a href=""https://www.google.com/aclk?..."
...,...,...,...,...
59,"<img src=""https://encrypted-tbn1.gstatic.com/s...","R$26,091.00",Casas Bahia,"<a href=""https://www.google.com/aclk?sa=l&ai=D..."
60,"<img src=""https://encrypted-tbn0.gstatic.com/s...","R$189,991.00",Zema,"<a href=""https://www.google.com/aclk?sa=l&ai=D..."
61,"<img src=""https://encrypted-tbn1.gstatic.com/s...","R$34,490.00",Amazon.com.br,"<a href=""https://www.google.com/aclk?sa=l&ai=D..."
62,"<img src=""https://encrypted-tbn0.gstatic.com/s...","R$228,905.00",Magazine Luiza,"<a href=""https://www.google.com/aclk?sa=l&ai=D..."
